# Gaussian Naive Bayes

https://www.youtube.com/watch?v=H3EjCKtlVog

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import scipy.stats as stats
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TODO: I somehow want to have a portfolio with these files. So how can I move the
#       data to another folder? Should I use my own Git repo and run using Jupyter?

data_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/machine-learning/data/Breast_cancer_data.csv')
data_raw.head(10)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
5,12.45,15.70,82.57,477.1,0.12780,0
6,18.25,19.98,119.60,1040.0,0.09463,0
7,13.71,20.83,90.20,577.9,0.11890,0
8,13.00,21.82,87.50,519.8,0.12730,0
9,12.46,24.04,83.97,475.9,0.11860,0


# Correlations

Naive Bayes algorithms perform well on datasets with no correlations in independent features. Removing the correlated features may improve the performance of the algorithm.

Source: https://www.analyticsvidhya.com/blog/2023/01/naive-bayes-algorithms-a-complete-guide-for-beginners/

In [ ]:
data_raw.corr()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
mean_radius,1.000000,0.323782,0.997855,0.987357,0.170581,-0.730029
mean_texture,0.323782,1.000000,0.329533,0.321086,-0.023389,-0.415185
mean_perimeter,0.997855,0.329533,1.000000,0.986507,0.207278,-0.742636
mean_area,0.987357,0.321086,0.986507,1.000000,0.177028,-0.708984
mean_smoothness,0.170581,-0.023389,0.207278,0.177028,1.000000,-0.358560
diagnosis,-0.730029,-0.415185,-0.742636,-0.708984,-0.358560,1.000000


In [ ]:
data = data_raw[['mean_radius', 'mean_texture', 'mean_smoothness', 'diagnosis']]
data.head(5)

,mean_radius,mean_texture,mean_smoothness,diagnosis
0,17.99,10.38,0.11840,0
1,20.57,17.77,0.08474,0
2,19.69,21.25,0.10960,0
3,11.42,20.38,0.14250,0
4,20.29,14.34,0.10030,0


In [ ]:
data.corr()

,mean_radius,mean_texture,mean_smoothness,diagnosis
mean_radius,1.000000,0.323782,0.170581,-0.730029
mean_texture,0.323782,1.000000,-0.023389,-0.415185
mean_smoothness,0.170581,-0.023389,1.000000,-0.358560
diagnosis,-0.730029,-0.415185,-0.358560,1.000000


# Algorithm

In [ ]:
train, test = train_test_split(data, test_size=0.2)

In [ ]:
features = ['mean_radius', 'mean_texture', 'mean_smoothness']
label_column = 'diagnosis'
labels = train[label_column].unique()

In [ ]:
train_by_label = { label: train[train[label_column] == label] for label in labels }

assert(train_by_label[0].shape[0] + train_by_label[1].shape[0] == train.shape[0])

In [ ]:
total = train.shape[0]

prior = { label: train_by_label[label].shape[0] / total for label in labels }

assert(prior[0] + prior[1] == 1)

In [ ]:
dists = { label: {} for label in labels }

for label in labels:
  for f in features:
    col = train_by_label[label][f]
    dists[label][f] = stats.norm(col.mean(), col.std())

Avoiding underflow (when probabilities are too small) can be done with this trick:

https://en.wikipedia.org/wiki/Log_probability

In [ ]:
def classify(input):
  p = dict(prior)

  for label in labels:
    p[label] = np.log(p[label])

    for f in features:
      p[label] += np.log(dists[label][f].pdf(input[f]))

  return max(p, key=lambda k: p[k])

# Test

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
label_correct = test[label_column]
label_predicted = [classify(row) for _, row in test.iterrows()]

print(confusion_matrix(label_correct, label_predicted))
print(f1_score(label_correct, label_predicted))

[[30  5]
 [ 3 76]]
0.95
